# HACKATHON CHALLENGE

Will start by importing the neccessary packages that will be used for the data processing.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from string import punctuation
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

from imblearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import metrics

In [3]:
train = pd.read_csv('/kaggle/input/south-african-language-identification-hack-2022/train_set.csv')
test = pd.read_csv('/kaggle/input/south-african-language-identification-hack-2022/test_set.csv')

# EXPLORATORY DATA ANALYSIS

**Exploratory data analysis** is a crucial process in Data analysis that enables us to gain insights into the data. We try to understand the data in details before pre-processing. 

In [4]:
#lets preview both the test and train data
print(f'shape == {train.shape}')
train.head()

In [5]:
#lets preview the first 5 entries of the test data
print(f'shape == {test.shape}')
test.head(5)

In [6]:
#lets see our unique y variable
languages = list(np.unique(np.array(train.lang_id.to_list())))
languages

It looks like we have 11 unique languages that will train our model on

In [7]:
#wec then check how our y variable is distributed among the 11 languages
total_dist = train.groupby(by = 'lang_id').count()
total_dist

So its true that our we wont need any upsampling or downsampling staff as our data is normally distributed among the 11 languages

In [8]:
#we plot a line graph with the count of languages on y axis
total_dist.plot()
plt.title('line graph showing count of languages')
plt.show

In [9]:
plt.pie(total_dist.text, labels = total_dist.index)
plt.show()

From the above two graphs, we can tell that our data in normally distributed

In [10]:
#we then print the first few information of our training set
for i in train.index:
    if i < 5:
        print(train.loc[i].text)
   
    elif i >32995:
        print(train.loc[i].text)

Lets have a look at the test data

In [11]:
#we preview the first 20 elements on the text entries
test.head(20).text.to_list()

From the above data We can note the following:
> 1. There are punctuations to be dealt with here
> 2. There is a unique tag (<fn>) That needs special attention

In [12]:
#We collent the entries having fn tags for analysis
have_fn = []
for i in test.index:
    txt = test.loc[i].text
    if '<fn>' in txt:
        have_fn.append(txt)

In [13]:
#lets print the first two of those special cases
have_fn[:2]

In [14]:
#how many of those special cases are available
fn_total = len(have_fn)
fn_total

In [15]:
test_total = test.shape[0]
labels = ['have fn tag','lack fn tag']
values = [fn_total,(test_total-fn_total)]
plt.pie(values, labels=labels, colors = ['red','green'])
plt.show()

In [16]:
#lets get the fn total percentage
fail_percentage = np.round((fn_total/test_total)*100,2)
fail_percentage

We have 1.9 percent of our data having fn tags. This implies that our model will havean f1 score of 98.1% maximum.
To solve that kind of problem, will pass our test data through a funtion which will separate the entries with <fn> tag with those lacking the tag.

Will then handle those with the <fn> tag with special considerations

##### 

# FEATURE ENGENEERING

We create a funtion called train_processing to process our train data before feeding it into the model

In [17]:
def train_preprocessing(text):
    
    '''
    This functions cleans text from line breaks, URLs, numbers, etc.
    ''' 
    
    text = text.replace(',','')
    text = text.replace('.',' ')
    text = text.lower() #to lower case
    text = text.replace('\n', ' ') # remove line breaks
    text = text.replace('\@(\w*)', '') # remove mentions
    text = re.sub(r"\bhttps://t.co/\w+", '', text) # remove URLs
    text = re.sub(r'\#', '', text) # remove hashtags. To remove full hashtag: '\#(\w*)'
    text = re.sub(' +', ' ', text) # remove 1+ spaces
    text = re.sub("\n"," ",text)
    text =' '.join(text.split())

    
    return text

In [18]:
#we then apply the text processing to our train data
train['text'] = train['text'].apply(train_preprocessing)

In [19]:
#we devide our features into the x and y as follows
X = train['text']
y = train['lang_id']

##### 

# MODEL BUILDING

Will use train test split to devide our data into training and testing set. Will the try multiple models and then pick the one performing best on the testing data

In [20]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.01)

### MultinomialNB(Best model for this data set)

Will strart with the model of intrest aka MultinomialNB Then will also check other models and see how they also performs

In [21]:
# Creating a pipeline for the gridsearch
from sklearn.model_selection import GridSearchCV
param_grid = {'alpha': [0.047]}  # setting parameter grid

tuned_mnb = Pipeline([('tfidf', TfidfVectorizer(
                                                max_df=0.9,ngram_range=(1, 5), analyzer= 'char'
                                               )),
                      ('mnb', GridSearchCV(MultinomialNB(),
                                           param_grid=param_grid,
                                           cv=5,
                                           scoring='f1_weighted'))
                      ])

tuned_mnb.fit(X_train, y_train)  # Fitting the model

y_pred_mnb = tuned_mnb.predict(X_val)  # predicting the fit on validation set

print(classification_report(y_val, y_pred_mnb))


### Other models

In [22]:
# Random Forest Classifier
rf = Pipeline([('tfidf', TfidfVectorizer(max_df=0.9,min_df = 2,ngram_range=(1, 1), analyzer= 'word')),
               ('clf', RandomForestClassifier(max_depth=100, 
                                              n_estimators=100))])

# Naïve Bayes:
nb = Pipeline([('tfidf', TfidfVectorizer(max_df=0.9,min_df = 2,ngram_range=(1, 1), analyzer= 'word')),
               ('clf', MultinomialNB())])

# K-NN Classifier
knn = Pipeline([('tfidf', TfidfVectorizer(max_df=0.9,min_df = 2,ngram_range=(1, 1), analyzer= 'word')),
                ('clf', KNeighborsClassifier(n_neighbors=5,  
                                             p=2))])

# Logistic Regression
lr = Pipeline([('tfidf',TfidfVectorizer(max_df=0.9,min_df = 2,ngram_range=(1, 1), analyzer= 'word')),
               ('clf',LogisticRegression(C=1,  
                                         max_iter=1000))])
# Linear SVC:
lsvc = Pipeline([('tfidf', TfidfVectorizer(max_df=0.9,min_df = 2,ngram_range=(1, 1), analyzer= 'word')),
                 ('clf', LinearSVC())])

Fiting other models 

In [23]:
# Random forest 
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_val)

# Niave bayes
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_val)

# K - nearest neighbors
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_val)

# Linear regression
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_val)

# Linear SVC
lsvc.fit(X_train, y_train)
y_pred_lsvc = lsvc.predict(X_val)

In [24]:
print('Random forest classifier')
print(classification_report(y_val, y_pred_rf))

In [25]:
print('Naive Bayes classifier')
print(classification_report(y_val, y_pred_nb))

In [26]:
print('K-Nearest Neighbours classifier')
print(classification_report(y_val, y_pred_knn))

In [27]:
print('Logistic linear Regression')
print(classification_report(y_val, y_pred_lr))

In [28]:
print('Linear SVC')
print(classification_report(y_val, y_pred_lsvc))

# Conclussion

Even Though MultinomialNB is the best model of choice, Other models when properly tuned can also perform absolutely well

# Kaggle submission

****We wont just call .predict on our test data but rather, will do the following:
> 1. Will clean our test data
> 2. Will create a funtion to separate the the special case ie <fn> with the non-special case
> 3. Will create a funtion to predict the special case and will use  .predict to predict the non-special case

We create a function called find_k which will deal with the speacial cases of our test data

In [35]:
def find_k(m):
    if '.en.' in m:
        k = 'eng'
    elif '.af.' in m:
        k = 'afr'
    elif 'tso_' in m:
        k = 'tso'
    elif '.xh.' in m:
        k = 'xho'
    elif '_Xho' in m:
        k = 'xho'
    elif 'Venda.' in m:
        k = 'ven'
    elif 'ndebele' in m.lower():
        k = 'nbl'
    elif 'sesotho' in m.lower():
        k = 'sot'
    elif 'sepedi' in m.lower():
        k = 'nso'
    elif 'zulu' in m.lower():
        k = 'zul'
    elif 'isiZulu.' in m:
        k = 'zul'
    elif 'isiXhosa.' in m:
        k = 'xho'
    elif 'Tshivenda' in m:
        k = 'ven'
    elif 'Xhosa' in m:
        k = 'xho'
    else:
        value = tuned_mnb.predict([train_preprocessing(m)])
        value = str(value)[2:5]
        k = value

    return k

In [37]:
def predict_test(data):
    if '<fn>' in data:
        value = find_k(data)
    else:
        value = tuned_mnb.predict([train_preprocessing(data)])
        value = str(value)[2:5]
        
    return value

We allow every x values to pass through the filter above ie prediction test which will sepate the special cases data with the normal data which our model can handle directly.

In [38]:
predictions = []
for i in test.index:
    txt = test.loc[i].text
    predictions.append(predict_test(txt))

We then create a csv file containing our predictions

In [39]:
# test['text'] = test['text'].apply(train_preprocessing)
submission_trial = pd.DataFrame(test['index'])
submission_trial['lang_id'] = predictions
submission_trial.to_csv('zf.csv', index=False)